In [1]:
import plaidml.keras
import os
plaidml.keras.install_backend()

In [2]:
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [3]:
import keras
import keras.backend as K

K

<module 'plaidml.keras.backend' from 'C:\\Users\\Bamouh Mohamed\\AppData\\Local\\conda\\conda\\envs\\Projectenv\\lib\\site-packages\\plaidml\\keras\\backend.py'>

In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.utils.vis_utils import model_to_dot
from keras.utils import to_categorical
import cv2
from pynput.keyboard import Key, Controller
from keras.models import model_from_json

In [6]:
kernel = np.ones((3,3),np.uint8)

In [31]:
pathTest = "fingers/test/*"
NUMBER_OF_CLASSES = 6
#indice_to_label = {0:"0L", 1:"1L", 2:"2L", 3:"3L", 4:"4L", 5:"5L", 6:"0R", 7:"1R", 8:"2R", 9:"3R", 10:"4R", 11:"5R"}
indice_to_label = {6:"None", 0:"0", 1:"1", 2:"2", 3:"3", 4:"4", 5:"5"}
label_to_indice = {v:k for k,v in indice_to_label.items()}

In [32]:
lower = np.array([0, 48, 80], dtype = "uint8")
upper = np.array([20, 255, 255], dtype = "uint8")

lower_gray = np.array([42], dtype = "uint8")
upper_gray = np.array([176], dtype = "uint8")

In [33]:
def process_image_from_video(img):
    img = cv2.resize(img, (128, 128))
    img = cv2.GaussianBlur(img,(5,5),0)
    _,img = cv2.threshold(img,20,255,cv2.THRESH_BINARY)
    #kernel = np.ones((3,3),np.uint8)
    #img = cv2.dilate(img, kernel, iterations = 1)
    #img = cv2.erode(img, kernel, iterations = 1)
    img = img/255
    return img

def process_image_for_training(img):
    img = cv2.resize(img, (128, 128))
    img = cv2.GaussianBlur(img,(5,5),0)
    _,img = cv2.threshold(img,80,255,cv2.THRESH_BINARY)
    img = img/255
    return img

def img_to_sample(img):
    img = img.astype(np.uint8)
    img = np.reshape(img, (128, 128))
    
    img = process_image_for_training(img)
    
    img = np.reshape(img, (128, 128, 1))
    return img

def loadModel(architecturePath="handmodel_fingers_architecture.json", weightsPath="handmodel_fingers_weights.hdf5"):
    # Model reconstruction from JSON file
    with open(architecturePath, 'r') as f:
        model = model_from_json(f.read())

    # Load weights into the new model
    model.load_weights(weightsPath)

    return model

In [11]:
model = loadModel(architecturePath="handmodel_fingers_architecture.json", weightsPath="handmodel_fingers_weights.hdf5")

INFO:plaidml:Opening device "opencl_amd_iceland.0"


In [39]:
lower = np.array([0, 48, 80], dtype = "uint8")
upper = np.array([20, 255, 255], dtype = "uint8")
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

def frame_to_hand_pose(gray, x1, y1, x2, y2, name):
    zone = gray[y1:y2, x1:x2, :]
    cv2.rectangle(gray,(x1,y1),(x2,y2),(0,255,0),3)
    
    zone = cv2.cvtColor(zone, cv2.COLOR_BGR2HSV)
    cv2.imshow('HSV zone {}'.format(name),zone)
    
    skinMask = cv2.inRange(zone, lower, upper)
    skinMask = cv2.dilate(skinMask, kernel, iterations = 2)
    skinMask = cv2.erode(skinMask, kernel, iterations = 2)
    skinMask = cv2.GaussianBlur(skinMask, (3, 3), 0)
    
    zone = cv2.bitwise_and(zone, zone, mask = skinMask)
    zone = cv2.cvtColor(zone, cv2.COLOR_BGR2GRAY)
    zone = process_image_from_video(zone)
    
    for_prediction = zone.reshape((1, 128, 128, 1))
    
    center = zone[64, 64]
    predicted_class = 6
    labels = ""
    
    if center != 0:
        preds = model.predict(for_prediction)
        predicted_class = np.argmax(preds)
        #labels = ["{} : {}%".format(k, format(preds[0][v]*100, '.2f')) for k,v in label_to_indice.items()]
    
    cv2.imshow('What the Network Sees {}'.format(name), zone)
    
    return center, predicted_class, labels

In [44]:
cap = cv2.VideoCapture(0)
keyboard = Controller()

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.flip(frame,1)
    
    center_left, predicted_class_left, labels_left = frame_to_hand_pose(gray, 10, 100, 260, 350, "left")
    center_right, predicted_class_right, labels_right = frame_to_hand_pose(gray, 400, 100, 650, 350, "right")
    
    if center_left == 1.0:
        keyboard.press(Key.left) 
        None
    elif center_right == 1.0:
        keyboard.press(Key.right)
        None
    else:
        keyboard.release(Key.right)
        keyboard.release(Key.left)
        None
        
    if predicted_class_left == 5 or predicted_class_left == 4 or predicted_class_right == 5 or predicted_class_right == 4:
        keyboard.press(Key.space)
        #keyboard.release(Key.space)
        None
    else:
        keyboard.release(Key.space)
        None
    
    y = 100
    cv2.putText(gray,indice_to_label[int(predicted_class_left)],(270,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2,cv2.LINE_AA)
    if labels_left != "":
        for label in labels_left:
            y = y + 20
            cv2.putText(gray, label,(270, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2,cv2.LINE_AA)
        
    yy = 250
    cv2.putText(gray,indice_to_label[int(predicted_class_right)],(270,yy), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2,cv2.LINE_AA)
    if labels_right != "":
        for label in labels_right:
            yy = yy + 20
            cv2.putText(gray, label,(270, yy), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2,cv2.LINE_AA)

    # Display the resulting frame      
    cv2.imshow('frame', gray)
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break                                                                                                                     
                                                                   
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()